# Реализации базы знаний для экспертной системы дифференциальной диагностики офтальмологических заболеваний

## 2. Процесс работы с данными

#### 2.1. Установка и подключение python-библиотек

Утановим необходимые python-библиотеки

In [12]:
!pip install networkx pyvis


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


Подключим нужные библиотеки:

In [13]:
import json
import os
import networkx as nx
from pyvis.network import Network
from IPython.display import JSON, display

### 2.2. Описание схемы данных базы знаний

База знаний представляет собой json-файл со следующей схемой

```
{
    "name": "DecisionTreeSchema",
    "description": "Схема дерева решений для медицинской диагностики",
    "root_type": "DecisionNode",

    "types": {
        "DecisionNode": {
            "description": "Узел дерева решений содержащий вопрос и ветвления",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "Текст вопроса или диагноза",
                    "required": true
                },
                "yes": {
                    "type": ["DecisionNode", "null"],
                    "description": "Ветвь для положительного ответа",
                    "required": true
                },
                "no": {
                    "type": ["DecisionNode", "null"],
                    "description": "Ветвь для отрицательного ответа",
                    "required": true
                }
            }
        }
    },

    "validation_rules": {
        "terminal_node": "Узел является терминальным (диагнозом) если оба yes и no равны null",
        "question_node": "Узел является вопросом если хотя бы одна из ветвей yes или no не null"
    },

    "examples": {
        "question_node": {
            "text": "Есть ли перикорнеальная инъекция?",
            "yes": {"$ref": "DecisionNode"},
            "no": {"$ref": "DecisionNode"}
        },
        "terminal_node": {
            "text": "Ирит",
            "yes": null,
            "no": null
        }
    }
}
```

### 2.3. Выгрузка базы знаний

Выгрузим базу знаний из json-докумета:

In [24]:
with open(os.path.join(os.getcwd(),"data.json")) as f:
    data = json.load(f)

Выведем загруженную базу знаний на дисплей:

In [25]:
display(JSON(data))

<IPython.core.display.JSON object>

### 2.4. Создание объекта графа

Построим объект графа принятия решения из загруженной базы знаний для дальнейшего вывода. Для этого воспользуемся библиотекой [networkx](https://networkx.org/)

In [18]:
# Создание графа NetworkX
G = nx.DiGraph()
node_counter = 0

# Функция для рекурсивного построения графа
def build_graph(data, parent_id=None, edge_label=None):
    global node_counter

    if data is None or data.get('text') is None:
        return

    current_id = node_counter
    node_counter += 1

    # Добавляем узел
    node_text = data['text']
    is_diagnosis =  not data['yes'] and not data['no']

    G.add_node(current_id,
               label=node_text,
               type='diagnosis' if is_diagnosis else 'question',
               color='lightgreen' if is_diagnosis else 'lightblue')

    # Добавляем ребро от родителя, если есть
    if parent_id is not None:
        G.add_edge(parent_id, current_id,
                   label=edge_label,
                   color='green' if edge_label == 'Да' else 'red')

    # Рекурсивно обрабатываем ответы "yes" и "no"
    if data.get('yes'):
        build_graph(data['yes'], current_id, 'Да')
    if data.get('no'):
        build_graph(data['no'], current_id, 'Нет')

# Построение графа
build_graph(data)

### 2.5. Вывод графа

Сделаем интерактивную визуализацию граф. Для этого воспользуемся библиотекой [pyvis](https://pyvis.readthedocs.io/en/latest/). После запуска ячейке ниже, на дисплей будет выведено интерактивное представление графа принятия решений. Также в текущей директории появится файл `graph.html` с представлением графа принятия решений.

In [19]:
# Инициализация механизма визуализации
net = Network(
    notebook=True,
    cdn_resources='in_line',
    directed=True,
    height='900px',
    width='100%',
    bgcolor='#f8fbff',
    font_color='black',
)

# Считывание графа из объекта networkx
net.from_nx(G)

# Настройка отображения узлов
for node in net.nodes:
    node['shape'] = 'dot'
    node['size'] = 30
    node['font'] = {'size': 18, 'color': 'black', 'vadjust': 0}
    node['title'] = node['label']
    node['labelHighlightBold'] = True
    node['color'] = {'background': node['color'], 'border': '#2b7ce9'}

# Настройка отображения ребер
for edge in net.edges:
    edge['color'] = G.edges[edge['from'], edge['to']]['color']
    edge['title'] = G.edges[edge['from'], edge['to']]['label']
    edge['label'] = G.edges[edge['from'], edge['to']]['label']
    edge['font'] = {'size': 14, 'align': 'top'}
    edge['arrows'] = {'to': {'enabled': True, 'scaleFactor': 1.2}}

# Настройка всего графа в целом
net.set_options("""
{
"interaction": {
    "zoomView": true,
    "dragView": true,
     "hover": true
  },
  "edges": {
    "scaling": {
      "min": 1,
      "max": 10,
      "label": {
        "enabled": true
      }
    }
  },
  "physics": {
    "enabled": false,
    "hierarchicalRepulsion": {
      "springLength": 200,
      "nodeDistance": 220,
      "damping": 0.1
    }
  },
  "layout": {
    "hierarchical": {
      "enabled": true,
      "levelSeparation": 200,
       "direction": "LR",
      "sortMethod": "directed"
    }
  }
}
""")

# Вывод графа
net.show('graph.html')

graph.html


## 3. Источники знаний о предметной области

1. Киселёв, А. Ю. Дифференциальная диагностика воспалительных заболеваний глаз [Текст] / А. Ю. Киселёв // Вестник офтальмологии. — 2022. — Т. 138, № 4. — С. 45–52.

2. Бойко, Е. В. Алгоритмы диагностики заболеваний сетчатки [Текст] / Е. В. Бойко // Офтальмология. — 2021. — Т. 18, № 2. — С. 33–40.

3. Егоров, Е. А. Дифференциальная диагностика помутнений роговицы [Текст] / Е. А. Егоров // Офтальмохирургия. — 2020. — № 3. — С. 18–26.

4. Шамшинова, А. М. Возрастные изменения зрительного анализатора [Текст] / А. М. Шамшинова // Вестник офтальмологии. — 2019. — № 6. — С. 7–15.

5. Кацнельсон, Л. А. Современные подходы к диагностике глаукомы [Текст] / Л. А. Кацнельсон // Офтальмология. — 2023. — Т. 20, № 1. — С. 12–20.

6. Михайлов, А. П. Воспалительные заболевания переднего отдела глаза у детей [Текст] / А. П. Михайлов // Российская детская офтальмология. — 2021. — № 2. — С. 45–51.

7. Фёдоров, С. Н. Оперативные методы лечения катаракты [Текст] / С. Н. Фёдоров // Офтальмохирургия. — 2018. — № 5. — С. 3–10.

8. Ченцов, В. В. Блефарит: диагностика и лечение [Текст] / В. В. Ченцов // Вестник офтальмологии. — 2020. — № 2. — С. 41–48.

9. Селиванов, П. А. Роговичные дистрофии: классификация и подходы к терапии [Текст] / П. А. Селиванов // Офтальмология. — 2022. — № 5. — С. 63–71.

10. Гордеев, Д. В. Нейроофтальмологические синдромы [Текст] / Д. В. Гордеев // Вестник офтальмологии. — 2021. — № 3. — С. 55–62.

11. Иванова, И. Н. Поражения сетчатки при системных заболеваниях [Текст] / И. Н. Иванова // Офтальмология. — 2020. — № 1. — С. 47–56.

12. Чухраева, Н. А. Современные представления о кератитах [Текст] / Н. А. Чухраева // Офтальмохирургия. — 2023. — № 4. — С. 9–18.

13. Орлова, Е. П. Особенности офтальмопатологии у пожилых пациентов [Текст] / Е. П. Орлова // Вестник офтальмологии. — 2019. — № 5. — С. 22–28.

14. Громов, С. А. Диагностика и лечение увеитов [Текст] / С. А. Громов // Офтальмология. — 2021. — № 6. — С. 30–38.

15. Петрова, Л. В. Поражения глаз при вирусных инфекциях [Текст] / Л. В. Петрова // Российская детская офтальмология. — 2022. — № 1. — С. 58–66.

16. Сидоров, Ю. А. Компьютерное моделирование офтальмологических диагнозов [Текст] / Ю. А. Сидоров // Офтальмохирургия. — 2020. — № 2. — С. 49–55.

17. Зорин, А. В. Дифференциальная диагностика катаракты и помутнений стекловидного тела [Текст] / А. В. Зорин // Вестник офтальмологии. — 2023. — № 1. — С. 27–35.

18. World Health Organization. International Classification of Diseases, 11th Revision (ICD-11) [Электронный ресурс] — URL: https://icd.who.int/en
 (Дата обращения: 09.10.2025).

19. American Academy of Ophthalmology. Preferred Practice Pattern® Guidelines [Электронный ресурс] — URL: https://www.aao.org/education/preferred-practice-patterns
 (Дата обращения: 09.10.2025).

20. National Eye Institute (NEI). Eye Conditions and Diseases [Электронный ресурс] — URL: https://www.nei.nih.gov/learn-about-eye-health/eye-conditions-and-diseases
 (Дата обращения: 09.10.2025).